# Matemáticas para Inteligencia Artificial (I)




### 1. Normas, distancias y ángulos




Sea $V$ un $\mathbb{R}$-espacio vectorial donde se ha definido un producto escalar $\langle\cdot,\cdot\rangle$. Recordemos que para que $\langle \cdot , \cdot\rangle$ sea un producto escalar, debe cumplir las siguientes propiedades: 

- $\langle x,y\rangle=\langle y,x\rangle, \; \forall\, x,y\in V$
- $\langle \alpha x + \beta y,z\rangle=\alpha \langle x,z\rangle+\beta\langle y, z\rangle, \; \forall\, x,y,z\in V$, $\forall\, \alpha,\beta\in\mathbb{R}$
- $\langle x,x\rangle \geq 0$, $\forall\, x\in V$.
- $\langle x,x\rangle =0$ si y sólo si $x=0$. 


Si se ha definido un producto escalar en $V$, entonces podemos calcular:

- normas de elementos $v\in V$ como $||v||:=\sqrt{\langle v,v\rangle}$
- distancia entre dos elementos $u,v\in V$ como $d(u,v):=||u-v||$
- ángulo $\alpha$ que forman dos elementos $u,v \in V$ a través de $\operatorname{cos}(\alpha):=\dfrac{|\langle u,v\rangle|}{||u||\cdot||v||}\in[-1,1]$

Veamos dos maneras alternativas de calcular el producto escalar de dos vectores en $V=\mathbb{R}^3$.

In [1]:
import numpy as np
from numpy import linalg as la

u=np.array([[1,0,1]])
v=np.array([[1],[2],[3]])

print("u =", u, "\n")
print("v =", v)

u = [[1 0 1]] 

v = [[1]
 [2]
 [3]]


In [ ]:
print(np.sum(u*v.T), "\n")
print(u@v)

In [ ]:
print("La distancia entre u y v es", la.norm(u-v.T))
print("El ángulo que forman u y v es", (np.abs(u@v)/(la.norm(u)*la.norm(v)))[0][0])

Además de la distancia euclídea, existen otros métodos para investigar cuándo dos vectores de $\mathbb{R}^n$ son _cercanos_ o _similares_. Por ejemplo, cuando la longitud de los vectores no es importante, la **similitud del coseno** resulta muy útil. Esto ocurre por ejemplo en _text mining_. 


$$ \text{sim}_{(u, v)}=\dfrac{\langle u, v\rangle}{||u|| ·||v||}=\text{cos}(\text{ángulo}(u, v))\in[-1,1],$$ 


<img src=https://lh6.googleusercontent.com/ycXLRE6YUiFrVJpkYXJe8I7oCiuUSYLhfBHvn81N3_AARdiEuswYKLqC5mLNqdqTiAkfCN7hBBdrIgQi6OAbOshJE1d3q_0XuWjik_KaQqryrh63PJiS9wDwT0M0NZ_AbOFpvEZgcMjbgYBXtM3VVdM width="1000">


A modo de ejemplo, veamos la _similitud del coseno_ entre los dos vectores anteriores.

In [ ]:
print(np.abs((u@v)[0][0])/(la.norm(u)*la.norm(v)))

Veamos ahora cuán similares son $v$ y el vector $(1,2,4)$.

In [ ]:
w=np.array([[1,2,4]])
print(np.abs((w@v)[0][0])/(la.norm(v)*la.norm(w)))

En este contexto, también se pueden definir otras distancias, como por ejemplo:


<img src=https://miro.medium.com/v2/resize:fit:1400/1*vAtQZbROuTdp36aQQ8cqBA.png width="800">


La célebre **distancia de Manhattan**, que se define como $d(u,v):=\displaystyle\sum_i |u_i-v_i|$, aplicada a $u$ y $v$, y a $v$ y $w$ proporcionaría:

In [ ]:
print(np.sum(np.abs(u-v.T)))
print(np.sum(np.abs(w-v.T)))

Por tanto, lo _cercanos_ que sean dos vectores/objetos depende de la distancia que se considere.

### 2. Algoritmos de recomendación


Un **sistema de recomendación** es un algoritmo que nos permite dar predicciones de cuál es el producto o ítem más adecuado para un usuario. Los sistemas de recomendación pueden ser de varias clases según el algoritmo utilizado, aunque solo nos centraremos en la presente práctica en los de _filtrado colaborativo_. 


Los sistemas de recomendación basados en algoritmos de **filtrado colaborativo** utilizan las valoraciones o interacciones de los usuarios sobre ciertos elementos del conjunto de productos, con el objetivo de predecir valoraciones o interacciones en el resto de los elementos y recomendar los de mayor valoración predicha. Ejemplos en esta línea pueden ser **Netflix**, **Spotify**, **Youtube**, etc.


Los datos iniciales de los que disponemos son una base de datos de usuarios o clientes $(user_1, user_2, \ldots, user_M)$ a los que vamos a recomendar una serie de productos o ítems $(ítem_1, ítem_2,\ldots, ítem_N)$, y las interacciones o puntuaciones de esos usuarios sobre algunos de los ítems, matriz $L$ de dimensión $M\times N$. 


**Ejemplo.** Supongamos que 10 usuarios están buscando en la página de **Ryanair** en el último mes vuelos desde Madrid (MAD) a Málaga (AGP), Las Palmas de Gran Canaria (LPA), Sevilla (SVQ), Valencia (VLC), Ibiza (IBZ) y Santiago de Compostela (SCQ). Supongamos que la fila $i$ del fichero `vuelos.csv` representa el número de búsquedas que ha realizado el usuario $i$ sobre cada una de las rutas antes mencionadas (en el orden citado). 

In [ ]:
import pandas as pd

vuelos = pd.read_csv('/Users/vmos/Library/CloudStorage/OneDrive-UPV/Curso IA (Samsung)/Apuntes VS (04.2024)/vuelos.csv')

In [ ]:
vuelos.head()

In [ ]:
V=vuelos.to_numpy()
print(V)

**Ejercicio.** A vista de los datos anteriores:
- ¿qué vuelo le recomendarías a los usuarios 1 y 2? 
- ¿Y a los usuarios 3, 5, 7 y 10? 


Dos filtrados colaborativos célebres son:


1. User-user: personas con intereses similares en el pasado es probable que tengan intereses similares en el futuro. Por tanto, a la hora de recomendar a un usuario un ítem, nos fijamos en los intereses en los que se han interesado usuarios similares a él .


**Ejemplo.** Messi ha visto Juego de Tronos y Breaking Bad. Cristiano ha visto Juego de Tronos, Breaking Bad y Vikingos. El algoritmo detecta que Messi y Cristiano son usuarios similares, luego el sistema recomienda Vikingos a Messi.


2. Ítem-ítem: si a un usuario le ha interesado en el pasado un producto, es probable que en el futuro le interesen productos similares. Por tanto, a la hora de recomendar a un usuario un ítem, nos fijamos en ítems similares a los que se ha interesado en el pasado.


**Ejemplo.** Messi, Cristiano, Haaland y Mbappé han visto Origen. Messi, Cristiano y Haaland también han visto Shutter Island. El algoritmo detecta que Origen y Shutter Island son productos similares, luego el sistema recomienda a Mbappé ver Shutter Island.


Queda de manifiesto, por tanto, que la clave es conocer la similitud entre usuarios o ítems, siendo la opción más "manejable" aquella que tenga menos elementos. Observemos que no estamos interesados en la longitud de los vectores, sino simplemente en su similitud, luego resulta natural utilizar la similitud del coseno.



En nuestro ejemplo vamos a realizar un algoritmo de recomendación ítem-ítem con el objetivo de recomendar a cada usuario un vuelo y después responderemos de nuevo a las preguntas anteriores. Comencemos analizando la similitud del resto de vuelos con el vuelo 6, pues es el que más veces ha buscado el usuario 1.

In [ ]:
print("sim(vuelo_1, vuelo_6) = ", (V[:,0:1].T@V[:,5:6])[0][0]/(la.norm(V[:,0:1])*la.norm(V[:,5:6])))

In [ ]:
print("sim(vuelo_5, vuelo_6) = ", (V[:,4:5].T@V[:,5:6])[0][0]/(la.norm(V[:,4:5])*la.norm(V[:,5:6])))

Ahora hagámoslo en general, es decir, calculemos las similitudes entre todos los vuelos entre sí. Esto nos derá una matriz $S$ de similitud entre vuelos, que ha de ser simétrica y con unos en la diagonal.

In [ ]:
S=(V/la.norm(V, axis=0)).T@(V/la.norm(V, axis=0))
print(S)

Sabiendo que el usuario 1 ha buscado previamente los vuelos 3 y 6 (una y dos veces, respectivamente), necesitamos buscar un vuelo que sea lo más parecido posible a los vuelos 3 y 6 (y preferiblemente más al 6 que al 3, es decir, **debemos ponderar estas búsquedas**). Por tanto, si multiplicamos (producto escalar) cada fila de $S$ por la primera fila de $V$ obtendremos unos valores que representan la _recomendación_ de los vuelos. Por tanto, bastará con quedarse con el valor más alto.

In [ ]:
V[0:1,:]@S

Ahora solamente tenemos que ver el valor más alto de los anteriores y que corresponda con un cero de la primera fila de $V$ (i.e. con un vuelo que aún no haya buscado). En este caso, sería el vuelo 4. Responde a las preguntas anteriores nuevamente fijándote en la siguiente matriz.

In [ ]:
print(np.round(V@S, 3))
print(V)

Trabajemos ahora con otro ejemplo de mayores dimensiones. Concretamente, la siguiente base de datos contiene puntuaciones de usuarios sobre ciertas películas.

In [ ]:
import pandas as pd

pelis = pd.read_csv('/Users/vmos/Library/CloudStorage/OneDrive-UPV/Curso IA (Samsung)/Apuntes VS (04.2024)/ratings_example.csv')
pelis.head()

Lo primero que debemos hacer en este caso es construirnos la matriz que nos interesa, donde las filas sean los usuarios y las columnas sean las películas.

In [ ]:
pelis_matriz=pelis.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
pelis_matriz.head()

In [ ]:
M=pelis_matriz.to_numpy()
print(M ,"\n", M.shape)

Por tanto, tenemos 671 usuarios que han valorado 9066 películas (observemos que hay etiquetas correspondientes a películas que no han sido vistas por nadie, como por ejemplo las que hay entre la 162673 y la 163948, luego no aparecen como columnas). 


En este caso, tenemos menos usuarios que películas, luego nos interesa ver la similitud entre usuarios.

In [ ]:
print("sim(user_1, user_2) = ", (M[0:1,:]@M[1:2,:].T)[0][0]/(la.norm(M[0:1,:])*la.norm(M[1:2,:])))

In [ ]:
print("sim(user_1, user_4) = ", (M[0:1,:]@M[3:4,:].T)[0][0]/(la.norm(M[0:1,:])*la.norm(M[3:4,:])))

Ahora hagámoslo en general, es decir, calculemos las similitudes entre todos los usuarios.

In [ ]:
S=(M.T/la.norm(M.T, axis=0)).T@(M.T/la.norm(M.T, axis=0))
print(S)

Si multiplicamos ahora el primer vector fila de $S$ por cada una columna de $M$, obtendremos un número que indica la _recomendación_ de esa película para el usuario 1. Y, en general, si multiplicamos por todas las columnas de $M$ y nos quedamos con el valor más alto (de aquellas películas que no ha visto el usuario 1), esa será nuestra recomendación.

In [ ]:
S[0:1,:]@M

In [ ]:
r=np.argmax(S[0:1,:]@M)
print(r)
print((S[0:1,:]@M)[0][r])
print((S[0:1,:]@M)[0][r]==np.max(S[0:1,:]@M))

In [ ]:
print(M[0,r])

In [ ]:
recomen=(S[0:1,:]@M)[0]
print(recomen)

recomen[M[0]!=0]=0
print(recomen)

# para asegurarnos de que coge el valor máximo de aquellas películas que el usuario no ha visto

r=np.argmax(recomen)
print(r)    

# seleccionamos ahora el mayor valor, y entonces seguro que no la ha visto

Luego le recomendamos la película/columna 232 de $M$, que es la película con Id 260:

In [ ]:
peliculas=pelis.movieId.to_numpy()

print("peliculas =", peliculas)

In [ ]:
pelis_matriz.columns[r]

Si queremos ahora saber qué recomendación haríamos a cada usuario, basta con hacer lo anterior en general: multiplicamos $S$ por $M$ y ordenaríamos los elementos de la misma forma anterior.

In [ ]:
K=S@M

In [ ]:
def Recomendacion(i):
    K[i-1][M[i-1]!=0]=0
    return np.argmax(K[i-1])

In [ ]:
for i in range(671):
    print("Al usuario", i+1, "le recomendamos la película", np.unique(peliculas)[Recomendacion(i+1)], "\n")